In [4]:
import os
import sys
import json
import glob
import copy
import collections
import re
import numpy as np
import importlib

sys.path.append('/code_location/multi_gpu')
import functions_parameter_handling
importlib.reload(functions_parameter_handling)


<module 'functions_parameter_handling' from '/code_location/multi_gpu/functions_parameter_handling.py'>

In [5]:
regex_model_dir = '/saved_models/arch_search_v02_topN/PND*/arch_*/'
list_model_dir = sorted(glob.glob(regex_model_dir))

for model_dir in list_model_dir:
    config_filename = os.path.join(model_dir, 'config.json')
    new_output_directory = model_dir
    functions_parameter_handling.migrate_config_to_new_output_directory(
        config_filename,
        new_output_directory,
        force_overwrite=True)

# len(list_model_dir)


Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0083/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0083/brain_arch.json
Copying files: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0083/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0083/brain_arch.json
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0083/pckl_network.pckl --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0083/pckl_network.pckl
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0083/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_fi

Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0288/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0288/brain_arch.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0288/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0288/config.json
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalHPv00/arch_0302/brain_arch.json
Copying files: /saved_models/arch_search_v02_topN/sr20000_cf

Copying files: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0154/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0154/brain_arch.json
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0154/pckl_network.pckl --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0154/pckl_network.pckl
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0154/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0154/brain_arch.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0154/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PAR

Copying files: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed0/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0302_seed0/brain_arch.json
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed0/pckl_network.pckl --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0302_seed0/pckl_network.pckl
Renaming in CONFIG: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed0/brain_arch.json --> /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0302_seed0/brain_arch.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_neg10pos10_filter_signalLPv01/arch_0302_seed0/config.json
config_dict similarity check is ig

Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_pos10pos30/arch_0191_seed0/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_pos10pos30/arch_0191_seed1/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_pos10pos30/arch_0191_seed1/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_pos10pos30/arch_0191_seed2/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_pos10pos30/arch_0191_seed2/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_

Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0154/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0190/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0190/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0191/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0191/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0191_seed0/config.json
config

config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0338/config.json
Pre-existing config file found: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0346/config.json
config_dict similarity check is ignoring: ['BRAIN_PARAMS/config', 'BRAIN_PARAMS/save_pckl_path', 'BRAIN_PARAMS/save_arch_path']
Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/PND_v08_noise_TLAS_snr_posInf/arch_0346/config.json


In [2]:
model_dir_format = '/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed{}'

basename_config = 'config.json'
basename_arch = 'brain_arch.json'

for new_idx in range(1,10):
    ref_model_dir = model_dir_format.format(0)
    new_model_dir = model_dir_format.format(new_idx)
    
    # Load REF_CONFIG and modify paths and random seed to make NEW_CONFIG
    ref_config_fn = os.path.join(ref_model_dir, basename_config)
    REF_CONFIG = functions_parameter_handling.load_config_dict_from_json(ref_config_fn)
    NEW_CONFIG = json.loads(json.dumps(REF_CONFIG).replace(os.path.basename(ref_model_dir),
                                                           os.path.basename(new_model_dir)))
    NEW_CONFIG['random_seed'] = REF_CONFIG['random_seed'] + new_idx
    
    # Load REF_ARCH
    ref_arch_fn = os.path.join(ref_model_dir, basename_arch)
    with open(ref_arch_fn, 'r') as f:
        REF_ARCH = json.load(f)

    new_config_fn = os.path.join(new_model_dir, basename_config)
    new_arch_fn = os.path.join(new_model_dir, basename_arch)
    if not os.path.exists(new_model_dir):
        os.mkdir(new_model_dir)
#     else:
#         raise ValueError('new_model_dir already exists!!!')
    
    functions_parameter_handling.write_config_dict_to_json(NEW_CONFIG,
                                                           new_config_fn,
                                                           force_overwrite=False)
    with open(new_arch_fn, 'w') as f:
        json.dump(REF_ARCH, f, sort_keys=True)
    
    print('Generated new model directory:\n\t{}\n'.format(new_model_dir))
    
#     print(json.dumps(NEW_CONFIG, indent=4, sort_keys=True))
#     print(json.dumps(NEW_ARCH, indent=4, sort_keys=True))


Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed1/config.json
Generated new model directory:
	/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed1

Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed2/config.json
Generated new model directory:
	/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed2

Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed3/config.json
Generated new model directory:
	/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0191_seed3

Successfully (over)wrote config file: /saved_models/arch_search_v02_topN/

In [3]:
print(json.dumps(NEW_CONFIG, indent=4, sort_keys=True))


{
    "BRAIN_PARAMS": {
        "batchnorm_flag": true,
        "config": "/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed9/brain_arch.json",
        "dropout_flag": true,
        "save_arch_path": "/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed9/brain_arch.json",
        "save_pckl_path": "/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed9/pckl_network.pckl",
        "trainable": true
    },
    "COCH_PARAMS": {},
    "EXTRAS": {
        "config_filename": "/saved_models/arch_search_v02_topN/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/arch_0302_seed9/config.json",
        "train_regex_brain": "/data/PND_v08/noise_TLAS_snr_neg10pos10/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/bez2018meanrates_0[0-7]*.tfrecords"
    },
    "ITERATOR_PARAMS": {
        "buffer": 6